## Configure Dataset and Dataloader

In [1]:
import torch
from torchvision import models
import torch.nn as nn
import torch.optim as optim

from utils.dataloader_image_classification import (
    ImageTransform, HymenopteraDataset, make_datapath_list
)
from utils.train import train_model

train_list = make_datapath_list(phase="train")
val_list = make_datapath_list(phase="val")

# Dataset
size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
train_dataset = HymenopteraDataset(train_list, transform=ImageTransform(size, mean, std), phase="train")
val_dataset = HymenopteraDataset(val_list, transform=ImageTransform(size, mean, std), phase="val")

# Dataloader
batch_size = 32

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True
)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False
)

dataloader_dic = {"train": train_dataloader, "val": val_dataloader}

.\data\hymenoptera_data\train\**\*.jpg
.\data\hymenoptera_data\val\**\*.jpg


## Network Model

In [2]:
# use pretrained model
use_pretrained = True
net = models.vgg16(pretrained=use_pretrained)

# change last classifer layer
net.classifier[6] = nn.Linear(in_features=4096, out_features=2)

# setup train mode
net.train()

print("Complete Network setup")

Complete Network setup


## Define Loss Function


In [3]:
criterion = nn.CrossEntropyLoss()

## Define Optimizer

* fine-tuning은 transfer learning과 optimizer 설정이 다름
* 모든 layer의 parameter를 재학습 할 수 있도록 설정 함 (Q: 이전 데이터로 학습한 지식이 지워지면 어떻게 하지? 이전 데이터로 학습한 지식이 완전히 없어지지 않고 새로운 데이터로 학습하는 내용과 잘 융합될 수 있도록 파라미터 설정해야 하는게 경험 지식인가?)
* features parameter, classifier 단계에서는 처음 두 개의 fully connected layer의 parameter, 교체한 fully connected layer(전결합층) parameter로 구분해서 각각 다른 학습률을 적용

In [4]:
params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

update_param_name_1 = ["features"]
update_param_name_2 = ["classifier.0.weight", "classifier.0.bias",
                       "classifier.3.weight", "classifier.3.bias"]
update_param_name_3 = ["classifier.6.weight", "classifier.6.bias"]

# save each parameter to list
for name, param in net.named_parameters():
    # set the flag to true so that model can be trained with new data
    param.requires_grad = True
    if update_param_name_1[0] in name:
        params_to_update_1.append(param)
        print(f"params_to_update1에 저장: {name}")
    elif name in update_param_name_2:
        params_to_update_2.append(param)
        print(f"params_to_update2에 저장: {name}")
    elif name in update_param_name_3:
        params_to_update_3.append(param)
        print(f"params_to_update3에 저장: {name}")
    else:
        param.requires_grad = True

# setup optimizer parameters
optimizer = optim.SGD([
    {'params': params_to_update_1, 'lr': 1e-4},
    {'params': params_to_update_2, 'lr': 5e-4},
    {'params': params_to_update_3, 'lr': 1e-3}
], momentum=0.9)

params_to_update1에 저장: features.0.weight
params_to_update1에 저장: features.0.bias
params_to_update1에 저장: features.2.weight
params_to_update1에 저장: features.2.bias
params_to_update1에 저장: features.5.weight
params_to_update1에 저장: features.5.bias
params_to_update1에 저장: features.7.weight
params_to_update1에 저장: features.7.bias
params_to_update1에 저장: features.10.weight
params_to_update1에 저장: features.10.bias
params_to_update1에 저장: features.12.weight
params_to_update1에 저장: features.12.bias
params_to_update1에 저장: features.14.weight
params_to_update1에 저장: features.14.bias
params_to_update1에 저장: features.17.weight
params_to_update1에 저장: features.17.bias
params_to_update1에 저장: features.19.weight
params_to_update1에 저장: features.19.bias
params_to_update1에 저장: features.21.weight
params_to_update1에 저장: features.21.bias
params_to_update1에 저장: features.24.weight
params_to_update1에 저장: features.24.bias
params_to_update1에 저장: features.26.weight
params_to_update1에 저장: features.26.bias
params_to_update1에 저장: f

## Train and Evaluate

In [5]:
num_epoch = 2
train_model(net, dataloader_dic, criterion, optimizer, num_epoch)

Epoch 1/2
--------------------------------------------------------------------------------


100%|██████████| 5/5 [00:40<00:00,  8.04s/it]


val Loss: 0.8455 Acc: 0.4314
Epoch 2/2
--------------------------------------------------------------------------------


100%|██████████| 8/8 [04:44<00:00, 35.52s/it]


train Loss: 0.5067 Acc: 0.7243


100%|██████████| 5/5 [00:46<00:00,  9.26s/it]

val Loss: 0.1640 Acc: 0.9412


## Save and Load model

In [7]:
# save model parameters
save_path = '.\\weights_fine_tuning.pth'
torch.save(net.state_dict(), save_path)

In [8]:
# load model parameters
load_path = '.\\weights_fine_tuning.pth'
load_weights = torch.load(load_path)
net.load_state_dict(load_weights)

<All keys matched successfully>